<div id="header" align="center">
    <h1>Rapport S6-APP4</h1>
    <br />
    <h2>GRO620 | Vision par ordinateur</h2>
    <br />
    <h3>Gabriel Cabana | cabg2101</h3>
    <h3>Olivier Roy | royo2206</h3>
</div>

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import random as rng
import matplotlib.pyplot as plt

print("GRO620 - Problématique")
print("OpenCV version", cv2.__version__)

%matplotlib inline

In [ ]:
## Si vous utilisez Google Colab, vous devez d'abord monter votre Google Drive
## où se trouve vos données. 
## Commentez les trois lignes suivantes en ajustant le chemin vers votre propre
## dossier :

#from google.colab import drive
#drive.mount('/content/gdrive')
#%cd /content/gdrive/MyDrive/BACC/S6/APP4

## Pour retrouver le chemin depuis Jupyter, vous pouvez utiliser ceci :
# !ls /content/gdrive/MyDrive

## 1. Introduction du problème

<br />
<div style="text-align: justify; text-indent: 50px">
Pour permettre de rendre possible le recyclage de quincaillerie utilisée dans les meubles préfabriqués d'une entreprise de mobilier, une solution est proposée. Il s'agit d'un système de vision permettant de détecter, classer et localiser les vis et boulons circulant sur un convoyeur. Pour démontrer l'efficacité de cette solution, une procédure de résolution du problème est présentée pour comprendre l'origine de la chaîne de traitement d'images utilisée. Une analyse des résultats est ensuite présentée permettant de facilement interpréter les résultats du système sur les images de démonstration utilisées.
</div>

***

## 2. Procédure de résolution

<br />
<div style="text-align: justify; text-indent: 50px">
Les objets qu'il faut détecter sont des vis sur une courroie de convoyeur en caoutchouc noir texturé. Comme les vis sont des objets longs et minces, certaines particularités doivent être prises en compte pour que la détection des pièces soit possible. Également, la surface de fond étant un caoutchouc noir texturé, il faut s'assurer que le traitement d'image permette d'éliminer le plus possible le bruit causé par les réflections de lumière et par la texture de la courroie.
</div>
<br />
<div style="text-align: justify; text-indent: 50px">
Les vis étant en acier réfléchissant, les réflections spéculaires de la source d'éclairage donne une couleur perçue par la caméra quasiment blanche par rapport au fond de l'image. Ça permet d'utiliser l'image sans faire de modifications au niveau du contraste ou de l'exposition.
</div>
    
### 2.1. Détection

<br />
<div style="text-align: justify; text-indent: 50px">
Pour détecter les pièces sur le convoyeur, il faut être capable de déterminer quels pixels dans l'image appartiennent à des vis. Il faut également être capable de regrouper les pixels appartenant à une vis avec les pixels adjacents qui appartiennent également à une vis pour créer des groupes correspondants à des objets détectés. En pratique, une série de traitements d'image est utilisée pour atteindre ce but.
</div>
<br />
<div style="text-align: justify; text-indent: 50px">
Le premier traitement qui devrait être appliqué est de convertir l'image couleur en nuances de gris. Cela permet de faire le traitement sur un seul canal de couleur et de simplifier le traitement. Comme l'arrière-plan n'est pas uniforme et cause des réflections de lumières indésirables, il est préférable de flouter l'image pour limiter le bruit. Pour séparer les pixels appartenant à l'arrière-plan des pixels appartenant aux vis, un seuillage (threshold) peut être appliqué permettant de binariser l'image. Si le seuillage est trop agressif, ce qui peut arriver lorsque les objets détectés sont assez minces, une vis pourrait séparée en deux morceaux dans l'image résultante ou carrément ignorée. Pour s'assurer que les vis soient détectées comme étant une seule entité, une transformation morphologique de dilatation peut être appliquée sur l'image. Cela permet d'enfler la surface occupée par les vis dans l'image et ainsi rejoindre les potentiels entités qui ont été séparées lors du seuillage. Un algorithme de détection de bordures dans l'image peut ensuite être appliqué pour faciliter la recherche de contours. Finalement, un algorithme de suivi de contours permet de discrétiser les différents contours dans l'image et d'obtenir une liste d'objets détectés.
</div>

In [ ]:
%%html

<!-- For web browser in dark mode (comment otherwise) -->
<style>
img {
    -webkit-filter: invert(100%);
    filter: invert(100%);
}
</style>

<div id="pipeline-diagram" align="center">
    <img src="./input/pipeline_diagram_cabg2101_royo2206.svg" alt="Diagramme du pipeline" width="100%"/>
    <h2> Figure 1 ― Diagramme du traitement du pipeline. </h2> 
</div>

### 2.2. Classement

<br />
<div style="text-align: justify; text-indent: 50px">
Une fois les vis détectées dans l'image, il est possible de les classer selon leur taille et de les identifier à l'aide d'un numéro. La recherche de contours ayant créé des groupes de pixels appartenant à un seul contour, il est maintenant plus facile de mesurer la taille des éléments détectés. Un numéro peut être assigné à chaque contour pour identifier l'objet. Si un rectangle est tracé en fonction des dimensions minimales d'un contour, celui-ci sera orienté dans la direction approximative de la vis. Les dimensions du rectangle représentent alors les dimensions de la vis et il est maintenant évident qu'il est possible de classer chaque vis selon sa longueur selon la taille du rectangle.
</div>

### 2.3. Localisation

<br />
<div style="text-align: justify; text-indent: 50px">
Pour localiser les différentes vis, la position sur le plan $\left(x,y\right)$ du convoyeur est recherchée ainsi qu'un angle entre 0 et 180°. Le rectangle de taille minimum tracé précédemment permet également d'obtenir la position de son centre et l'angle de la vis. Cependant, la position obtenue n'est pas dans le repère recherché. Il faut donc faire une projection des positions obtenus du repère de la caméra $\{C\}$ jusqu'au repère du convoyeur $\{0\}$ grâce aux informations intrinsèques et extrinsèques disponibles. Comme la matrice de transformation $T$ entre les deux repères est déjà fournie, il suffit de calculer la matrice de la caméra $P$ à partir de la matrice de calibration $K$ et de la matrice de transformation.
</div>

***

## 3. Mise en oeuvre du pipeline

<br />
<div style="text-align: justify; text-indent: 50px">
Après avoir expliqué la théorie derrière les différentes étapes de traitements requis pour faire la détection, la classification et la localisation de vis dans les images, voyons comment implémenter les méthodes en Python pour faire fonctionner le système.
</div>

### 3.1. Méthodes des objets

<br />
<div style="text-align: justify; text-indent: 50px">
Différentes fonctions ont été définies pour rendre la programmation plus modulaire et permettre de la rendre plus facile à interpréter. Les méthodes ci-dessous permettent de retourner les informations nécessaires ou de faire les vérifications concernant les items détectés.
</div>

- `itemType()`:  Retourne le type de vis selon sa longueur ;
- `itemAngle()`: Retourne l'angle de la vis en radians ;
- `itemInfo()`:  Retourne un dictionnaire contenant les informations pertinentes ;
- `itemMatch()`: Vérifie si un contour détecté est dans la plage de tolérance d'un contour détecté précédemment ;
- `itemProportions()`: Vérifie si les proportions d'un contour détecté sont valides.

In [ ]:
##################################################
### ITEM METHODS                               ###
##################################################

def itemType(size):
    # Return item type depending on its longer dimension 
    item_length = np.abs(size).max()
    
    if item_length < 100:
        item_type = "courte"
    else:
        item_type = "longue"
    
    return item_type

def itemAngle(angle, width, height):
    # Return item angle between 0 and pi radians.
    if(width < height):
        angle -= 90

    return angle * -np.pi/180

def itemInfo(position, size, theta, box):
    # Return all info about an item in a dictionary

    #Identify item type.
    item_type = itemType(size)
    
    # Change angle into a more intuitive value.
    theta = itemAngle(theta, size[0], size[1])
    
    return {"type": item_type, "x": position[0], "y": position[1], "w": size[0], "h": size[1], "angle": theta, "box": box}

def itemMatch(items, new_item):
    # Verify if a detected contour was already detected previously
    known_item = False
    pos_tol    = 10            # px
    new_x      = new_item["x"] # px
    new_y      = new_item["y"] # px

    for item in items:
        saved_x = item["x"] # px
        saved_y = item["y"] # px

        if abs(saved_x - new_x) < pos_tol and \
           abs(saved_y - new_y) < pos_tol:
            known_item = True
            
    return known_item

def itemProportions(size):
    # Verify that the maximum and minimum dimensions are correct
    width_max  = 50 # px
    length_min = 50 # px
    
    if size.max() < length_min or \
       size.min() > width_max:
        return False
    else:
        return True

### 3.2. Méthodes de traitement d'image

<br />
<div style="text-align: justify; text-indent: 50px">
Les méthodes utilisées pour faire le traitement d'image regroupe les fonctions permettant d'appliquer les traitements de façon séquentielle pour obtenir l'information souhaitée, mais aussi celles qui permettent de trier et afficher les contours détectés.
</div>
    
- `getImageContours()`:
    - Conversion de BGR en nuances de gris ;
    - Application d'un filtre gaussien pour flouter l'image ;
    - Application d'un seuillage pour binariser l'image ;
    - Application d'une transformation morphologique de dilatation pour amplifier les traits ;
    - Détection des bordures ;
    - Recherche de contours ;
    - Si souhaité, retour d'images montrant les étapes de traitement du pipeline.
- `sortImageContours()`:
    - Validation des contours détectés (proportions, proximité / superposition avec un contour détecté précédemment) ;
    - Tri des contours en fonction de leur position dans l'image.
- `drawImageContours()`:
    - Affichage des cadre de délimitation ainsi que des numéros d'identification sur l'image ;
    - Si souhaité, affichage des cadre de délimitation ainsi que des numéros d'identification sur la dernière étape de traitement du pipeline.

In [ ]:
##################################################
### IMAGE METHODS                              ###
##################################################

def getImageContours(image, get_pipeline=False):
    blur_dim      = 11
    threshold_min = 190
    threshold_max = 200
    dilation_dim  = 3
    canny_min     = 100
    canny_max     = 150
    
    # Convert from BGR to grayscale.
    img_mono = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blur using a Gaussian Filter.
    img_blur = cv2.GaussianBlur(img_mono, (blur_dim, blur_dim), 0)
    
    # Convert to binary with threshold.
    _, img_threshold = cv2.threshold(img_blur, threshold_min, threshold_max, cv2.THRESH_BINARY)
    
    # Dilate features to amplify contour detection.
    kernel = np.ones((dilation_dim, dilation_dim), np.uint8)
    img_dilated = cv2.dilate(img_threshold, kernel, iterations = 1)
    
    # Detect edges using Canny.
    img_edges = cv2.Canny(img_dilated, canny_min, canny_max)
    
    # Find contours.
    contours, hierarchy = cv2.findContours(img_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    if not get_pipeline:
        return contours, hierarchy, None
    else:
        pipeline = list()
        pipeline.append(img_mono)
        pipeline.append(img_blur)
        pipeline.append(img_threshold)
        pipeline.append(img_dilated)
        pipeline.append(img_edges)
        return contours, hierarchy, pipeline
    
def sortImageContours(contours):
    items = list()
    
    for contour in contours:
        # Get minimum area rectangle which stores position (x, y), size (width, height), and angle (theta)
        new_rect     = cv2.minAreaRect(contour)
        
        # Split data.
        new_position = np.asarray(new_rect[0]) # Position (x, y)
        new_size     = np.asarray(new_rect[1]) # Size (width, height)
        new_angle    = new_rect[2]             # Angle (theta)
        
        # Create item dictionary for further processing.
        new_item     = itemInfo(new_position, new_size, new_angle, np.int0(cv2.boxPoints(new_rect)))

        # If contour is new, save information.
        if not itemMatch(items, new_item) and itemProportions(new_size):
            items.append(new_item)
            
    # Sort items (up and down, left to right).
    items = sorted(items, key = lambda k: k["y"])
    items = sorted(items, key = lambda k: k["x"])
            
    return items
    
def drawImageContours(image, items, pipeline=None):
    for i, item in enumerate(items):
        color = (rng.randint(0,256), rng.randint(0,256), rng.randint(0,256))
        
        # Draw bounding box.
        cv2.drawContours(image, [item["box"]], 0, color, 2)
        
        # Identify bounding box.
        cv2.putText(image, str(i), (int(item["x"]), int(item["y"])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
    
        if pipeline is not None:
            # Draw bounding box on last pipeline step.
            cv2.drawContours(pipeline[-1], [item["box"]], 0, (128, 128, 128), 2)
            
            # Identify bounding box on last pipeline step.
            cv2.putText(pipeline[-1], str(i), (int(item["x"]), int(item["y"])), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

### 3.3. Méthodes de transformations

<br />
<div style="text-align: justify; text-indent: 50px">
Pour convertir les positions obtenues par les contours du repère de la caméra $\{C\}$ dans le repère du convoyeur $\{0\}$, certaines transformations et opérations matricielles sont nécessaires. Les positions obtenues par la position des contours sont sur la surface du capteur, en pixels. Il faut donc faire une projection sur le plan $\left(x,y\right)$ du repère du convoyeur. Les méthodes suivantes ont été utilisées pour faire les opérations permettant de faire la conversion de repère.
</div>

- `getTransformationMatrix()`: Retourne la matrice de transformation entre le repère de la caméra et le repère du convoyeur ;
- `getInverseCameraMatrix()`: Calcule l'inverse de la matrice de caméra en fonction des propriétés intrinsèques ;
- `getItemsWorldCoordinates()`: Retourne la position dans le repère du convoyeur à partir d'une position dans le repère de la caméra, en pixels.

In [ ]:
##################################################
### TRANSFORM METHODS                          ###
##################################################
            
def getTransformationMatrix():
    return np.array(([1,  0,  0, 0.500],
                     [0, -1,  0, 0.200],
                     [0,  0, -1, 0.282],
                     [0,  0,  0,     1]))
            
def getInverseCameraMatrix(transformation_matrix):
    sensor_pixel_width  = 640.0  # px
    sensor_pixel_height = 427.0  # px
    sensor_width        = 0.0234 # m
    sensor_height       = 0.0156 # m
    focal_length        = 0.0230 # m

    fx = focal_length / sensor_width  * sensor_pixel_width  # px
    fy = focal_length / sensor_height * sensor_pixel_height # px

    # Set calibration matrix.
    K = np.array([[fx,  0,  sensor_pixel_width/2],
                  [0,  fy, sensor_pixel_height/2],
                  [0,   0,                     1]])
    
    Kt = np.zeros((4,4))
    Kt[:-1,:-1] = K
    Kt[-1,-1]   = 1
    
    # Get camera matrix.
    Pt = Kt @ np.linalg.inv(transformation_matrix)
    
    return np.linalg.inv(Pt)
            
def getItemsWorldCoordinates(items, transformation_matrix, inverse_camera_matrix):
    for i, item in enumerate(items):
        pos_x = item["x"]
        pos_y = item["y"]
        
        # Transform screen coordinates into global position.
        coordinates_screen      = np.array([pos_x, pos_y, 1, 1/transformation_matrix[2][-1]])
        coordinates_reprojected = coordinates_screen / coordinates_screen[-1]
        coordinates_world       = inverse_camera_matrix.dot(coordinates_reprojected) 
        
        # Change item definition and content for export.
        items[i] = [item["type"], coordinates_world[0], coordinates_world[1], coordinates_world[2], item["angle"]]
        
    return items

### 3.4. Déverminage

Lorsque le mode de déverminage est activé, on peut afficher et exporter les étapes du traitement d'images du pipeline.

In [ ]:
##################################################
### DEBUGGING METHODS                          ###
##################################################

def displayPipeline(original, final, pipeline, index):
    simfig, plots = plt.subplots(4, 2, figsize = (6,10))
        
    plots[0][0].imshow(original)
    plots[0][0].axis("off")
    plots[0][0].set_title("1. Original")

    plots[0][1].imshow(pipeline[0], cmap="gray")
    plots[0][1].axis("off")
    plots[0][1].set_title("2. Monochrome")

    plots[1][0].imshow(pipeline[1], cmap="gray")
    plots[1][0].axis("off")
    plots[1][0].set_title("3. Filtre gaussien")

    plots[1][1].imshow(pipeline[2], cmap="gray")
    plots[1][1].axis("off")
    plots[1][1].set_title("4. Seuillage")

    plots[2][0].imshow(pipeline[3], cmap="gray")
    plots[2][0].axis("off")
    plots[2][0].set_title("5. Dilatation")

    plots[2][1].imshow(pipeline[4], cmap="gray")
    plots[2][1].axis("off")
    plots[2][1].set_title("6. Canny")

    plots[3][0].imshow(pipeline[5], cmap="gray")
    plots[3][0].axis("off")
    plots[3][0].set_title("7. Cadre de délimitation")

    plots[3][1].imshow(final)
    plots[3][1].axis("off")
    plots[3][1].set_title("8. Final")

    simfig.tight_layout()
    simfig.savefig(f"output/pipeline_{index+1:02}.pdf", dpi=300)

    plt.figure()
    plt.imshow(final)

### 3.5. Préambule du programme principal

<br />
<div style="text-align: justify; text-indent: 50px">
Cette section effectue l'initialisation des images, crée les répertoires nécessaires et vérifie la présences des fichiers requis pour faire le traitement. C'est à la deuxième ligne que l'on peut activer le mode de déverminage en passant le booléen correspondant <code>DEBUG</code> de <code>False</code> à <code>True</code>.
</div>

In [ ]:
# Set to "True" if you wish to see the pipeline output step by step.
DEBUG = False

# Create output directory for CSV files (and debug files).
if not os.path.exists("./output"):
    os.makedirs("./output")

# Verify that source files are available.
images = os.listdir("photos_prob/")
if (len(images) == 0):
    raise Exception("ERREUR! Vérifiez que vous avez bien un dossier photos_prob au même endroit que ce calepin.")

# Replace image path with image content.
for i, f in enumerate(images):
    img = cv2.imread(os.path.join("photos_prob/", f))
    images[i] = img

### 3.6. Programme principal

<br />
<div style="text-align: justify; text-indent: 50px">
Cette section du code appelle de façon séquentielle les fonctions permettant de faire la détection, la classification et la localisation des vis dans les images d'entrée. La détection des contours s'effectue d'abord, suivi par la transformation des positions du repère de la caméra jusqu'au repère du convoyeur. Les informations sont ensuite exportées dans un fichier CSV pour chaque image. 
</div>

In [ ]:
##################################################
### MAIN PROGRAM                               ###
##################################################

for i, image in enumerate(images):
    # Use processing pipeline to get contours.
    contours, _, pipeline = getImageContours(image, get_pipeline=DEBUG)
    
    # Find valid contours and sort items.
    items = sortImageContours(contours)
    
    # If debugging to get pipeline, make copy or original image and append last pipeline step before drawing.
    if DEBUG:
        image_original = image.copy()
        pipeline.append(pipeline[-1].copy())
    
    # Draw box and add identification number as text.
    drawImageContours(image, items, pipeline=pipeline)

    # We wish to find the screws positions projected onto the frame {0} (conveyor) from the frame {1} (camera).
    # The last function, "getItemsWorldCoordinates" will change items for dictionaries to lists before exporting.
    transformation_matrix = getTransformationMatrix()
    inverse_camera_matrix = getInverseCameraMatrix(transformation_matrix)
    items                 = getItemsWorldCoordinates(items, transformation_matrix, inverse_camera_matrix)
        
    # Data export (CSV).
    item_info = pd.DataFrame(items, columns = ["Type", "X (m)", "Y (m)", "Z (m)", u"\u03B8"+" (rad)"])
    item_info.index.name = "id"
    item_info.to_csv(f"output/image_{i+1:02}.csv", float_format="%.3f", encoding="utf-8-sig")
    
    # Show pipeline and save as PDF.
    if DEBUG:
        displayPipeline(image_original, image, pipeline, i)

## Analyse du résultat

<br />
<div style="text-align: justify; text-indent: 50px">
Pour valider que la chaîne de traitement d'images permettre de détecter, classer et localiser les vis dans les images d'entrée, il est important de valider visuellement que les étapes de traitements semblent produire les résultats attendus. Dans le code produit, il est possible d'activer le mode de déverminage (<code>DEBUG</code>) qui permet l'affichage et l'exportation complète d'images permettant de visualiser les étapes du traitement d'images. Le succès du système de vision pour détecter, classer et localiser les vis dans les images fournies a donc pu être validé et les résultats semblent démontrer que le système de vision permettrait d'accomplir le but recherché. Il est possible de voir ci-bas un example de chaque étape du traitement d'images pour bien visualiser leur importance.
</div>

In [ ]:
%%html

<div id="pipeline-example" align="center">
    <img src="./input/pipeline_example_cabg2101_royo2206.svg" alt="Exemple de la chaine de traitement d'images" width="100%"/>
    <h2> Figure 2 ― Exemple de la chaîne de traitement d'images. </h2>
</div>

<div style="text-align: justify; text-indent: 50px">
Dans la figure 2, il est possible de voir que le seuillage cause en effet les vis à s'amincir considérablement et même dans certain cas à les séparer en deux morceaux (comme dans le cas de la vis #9). C'est alors que l'on peut constater l'utilité de la transformation en dilatation pour assurer que chaque vis est reconnue comme étant une seule entité. L'algorithme de Canny de détection de bordures est capable de représenter de façon assez fidèle la forme de contour de chacune des vis. On peut finalement voir sur les deux dernières images que les rectangles de surface minimale permettent de localiser les vis mais aussi d'en déterminer la longueur et l'orientation.
</div>